In [ ]:
# Data

# GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device", device)

In [ ]:
# PMF
# N: # of users
# M: # of movies
# D: dimension

import torch
import torch.nn as nn
import torch.optim as optim


class PMF(nn.Module):
  def __init__(self, train_R, test_R, d, lambdaV, lambdaU, lambdaW,learning_rate=1e-3, epochs=200, constrain = False, device):
    '''
    constrain: True -> constrained pmf
    I: rating 있으면 1 없으면 0
    '''
    self.n, self.m = train_R.shape
    self.device = device
    self.latent_dimension = d
    self.U = nn.Parameter(torch.rand(self.n, self.latent_dimension).to(self.device)) # random initialize (D, N)
    self.V = nn.Parameter(torch.rand(self.m, self.latent_dimension).to(self.device)) # (D, M)
    self.constrain = constrain

    if self.constrain:
      self.W = nn.Parameter(torch.rand(self.m, self.latent_dimension)) # (D, M)

    self.lambdaV = lambdaV
    self.lambdaU = lambdaU
    self.lambdaW = lambdaW
    self.train_R = train_R
    self.test_R = test_R
    self.lr = learning_rate
    self.epoch = epochs
    self.I = (train_R > 0).int() # (N, M)

    self.optimizer = optim.Adam(self.parameters(), lr=self.lr)
    self.criterion = nn.MSELoss()

  def loss(self):
    if self.constrain:
      predicted_R = self.I * torch.matmul(self.U.transpose(0, 1), self.V + self.W)
      loss = 0.5 * self.criterion(predicted_R, train_R) + self.lambdaV / 2 * torch.sum(self.U**2)
    else:
      predicted_R = self.I * torch.matmul(self.U.transpose(0, 1), self.V)
      loss = 0.5 * self.criterion(predicted_R, train_R) + self.lambdaU / 2 * torch.sum(self.U**2) + self.lambdaV / 2 * torch.sum(self.V**2)
    return loss
